
---

# 📊 Proyecto Final: Inteligencia de Negocios y Minería de Datos

## 📘 Informe de Proyecto Final

**Materia:** Inteligencia de Negocios (ICC-321-T)
**Tema:** Dashboard Interactivo y Modelo de Minería de Datos Descriptivo

**Autores:**

* Randy Alexander Germosén Ureña *(1013-4707)*
* Fernando Almonte Delgado *(1015-7628)*

**Repositorio:**
[icc321-2025-final](https://github.com/TZeik/icc321-2025-final) <img src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" width="15" height="15"/>

---

## 🎯 Objetivo del Proyecto

El propósito de este proyecto es desarrollar una solución integral de Inteligencia de Negocios utilizando datos públicos del gobierno de la República Dominicana. Consta de dos componentes principales:

1. **Dashboard Interactivo:** Permite visualizar y monitorear métricas de gasto y nómina para apoyar la toma de decisiones.
2. **Modelo de Minería de Datos:** Implementación de un modelo descriptivo (Clustering) para descubrir patrones y segmentar perfiles de empleados.

---

## 📂 Datasets Utilizados

Se procesaron y unificaron datos históricos abarcando el periodo **2018–2025**:

1. **Nómina de la Contraloría General de la República:**
   Información detallada sobre empleados, cargos, departamentos y sueldos.
2. **Índice de Precios al Consumidor (IPC):**
   Datos del Banco Central utilizados para calcular el **salario real** (ajustado por inflación) en comparación con el salario nominal.

---

## 🧠 Metodología

El desarrollo del proyecto se estructuró en las siguientes fases técnicas:

### 1. Ingeniería de Datos (ETL)

* **Extracción y Limpieza:**

  * Unificación de múltiples archivos CSV mensuales/anuales.
  * Estandarización de nombres de cargos, normalización de formatos monetarios y corrección de codificación (`latin-1`, `utf-8`).
  * Homogeneización de los nombres de los meses.
* **Enriquecimiento:**

  * Cruce entre nómina e IPC para calcular la pérdida de poder adquisitivo.

### 2. Almacenamiento (Data Warehousing)

* Implementación de un **Data Warehouse** local con **SQLite**.
* Diseño bajo un **Esquema en Estrella**, con:

  * Tabla de hechos: `fact_nomina`
  * Tablas de dimensiones: `dim_empleado`, `dim_tiempo`

### 3. Visualización (Dashboard)

* Creación de Dashboard interactivo en **Tableau Public**.
* Diseño de KPIs como:

  * Gasto total,
  * Brecha salarial,
  * Evolución de plantilla,
  * Tendencias del salario real vs nominal.

### 4. Minería de Datos (Machine Learning)

* **Preprocesamiento:**
  Codificación de variables categóricas y escalado numérico.
* **Modelado:**
  Aplicación de **K-Means Clustering** para identificar grupos de empleados con características similares.
* **Evaluación:**

  * Método del Codo
  * Coeficiente de Silueta

---

## 📊 Resultados Principales

La solución permite analizar hallazgos relevantes como:

* Diferencias entre **Sueldo Nominal** y **Sueldo Real** a lo largo del tiempo.
* Identificación de departamentos con mayor incremento en el gasto de nómina.
* Clusters de empleados basados en sueldo, cargo y antigüedad, revelando patrones ocultos en la organización.

---

## 🧩 Herramientas Utilizadas

### Lenguajes y Entorno

* **Python 3.x** (Jupyter Notebook)

### Librerías Principales

* `pandas` — Manipulación y limpieza de datos
* `sqlite3` — Data Warehouse local
* `scikit-learn` — Algoritmo K-Means y métricas
* `matplotlib` — Visualización del método del codo

### Visualización

* **Tableau Public** — Dashboard interactivo final

---



In [1]:
import pandas as pd
import sqlite3
import numpy as np

### Configuración de Entorno y Reglas de Negocio
Importación de librerías y definición de funciones auxiliares para la limpieza de datos monetarios, estandarización de texto y cálculo estimado del Impuesto Sobre la Renta (ISR).

In [2]:
meses_map = {
    'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
    'JULIO': 7, 'AGOSTO': 8, 'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
}

meses_inv_map = {v: k for k, v in meses_map.items()}

def limpiar_moneda(valor):
    if pd.isna(valor): return 0.0
    s = str(valor).replace('RD$', '').replace(',', '').strip()
    try:
        return float(s)
    except:
        return 0.0

def limpiar_texto(texto):
    if pd.isna(texto): return "DESCONOCIDO"
    return str(texto).strip().upper()

def calcular_isr_estimado(sueldo_mensual):
    """
    Cálculo simplificado del ISR mensual (Escala aproximada DGII Rep. Dom).
    Exento hasta ~34,685.
    """
    if sueldo_mensual <= 34685:
        return 0.0
    elif sueldo_mensual <= 52027:
        excedente = sueldo_mensual - 34685
        return excedente * 0.15
    elif sueldo_mensual <= 72260:
        excedente = sueldo_mensual - 52027
        return 2601 + (excedente * 0.20)
    else:
        excedente = sueldo_mensual - 72260
        return 6648 + (excedente * 0.25)

print("Funciones y lógica de impuestos configuradas.")

Funciones y lógica de impuestos configuradas.


### Extracción, Transformación y Limpieza (ETL)
Carga de los datasets crudos (IPC y Nómina), normalización de nombres de columnas, conversión de tipos de datos y filtrado de registros nulos o inválidos para asegurar la calidad de los datos base.

In [3]:
# ---------------------------------------------------------
# CARGA Y TRANSFORMACIÓN INICIAL (ETL)
# ---------------------------------------------------------
# CARGA DE IPC
df_ipc = pd.read_csv('./raw_data/ipc_base_1984-2025.csv', sep=';')
df_ipc = df_ipc[df_ipc['PERIODO'] >= 2018].copy()

# Limpieza de Meses
df_ipc['MES_LIMPIO'] = df_ipc['MES'].apply(limpiar_texto).map(meses_map)

def clean_float(val):
    try:
        return float(str(val).strip())
    except:
        return 0.0

# Seleccionamos y limpiamos
cols_ipc = ['INDICE', 'VAR_MES', 'VAR_DIC', 'VAR_12', 'PROM_12']
for col in cols_ipc:
    df_ipc[col] = df_ipc[col].apply(clean_float)

# Preparamos el dataframe limpio de IPC
df_ipc_clean = df_ipc[['PERIODO', 'MES_LIMPIO'] + cols_ipc].rename(columns={
    'PERIODO': 'ANIO', 
    'MES_LIMPIO': 'MES_NUM',
    'INDICE': 'IPC',
    'VAR_MES': 'inflacion_mensual',
    'VAR_DIC': 'inflacion_acumulada',
    'VAR_12': 'inflacion_interanual',
    'PROM_12': 'ipc_promedio_12m'
})

# CARGA DE NÓMINA
df_nomina = pd.read_csv('./raw_data/nomina-empleados-fijos-y-contratados-CSV-2018-2025.csv', 
                        sep=';', encoding='latin-1', low_memory=False)

df_nomina.columns = (df_nomina.columns.str.strip().str.upper()
                     .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8"))
df_nomina.columns = [c.replace('AO', 'ANIO') for c in df_nomina.columns]

df_nomina['SUELDO_NOMINAL'] = df_nomina['SUELDO'].apply(limpiar_moneda)
df_nomina['ANIO'] = pd.to_numeric(df_nomina['ANIO'].astype(str).str.extract(r'(\d{4})')[0], errors='coerce')
df_nomina['MES_NUM'] = df_nomina['MES'].astype(str).apply(limpiar_texto).map(meses_map)

for col in ['NOMBRE', 'FUNCION', 'DEPARTAMENTO', 'ESTATUS']:
    df_nomina[col] = df_nomina[col].apply(limpiar_texto)

df_nomina['NOMBRE'] = df_nomina['NOMBRE'].str.replace('Ï¿½', 'Ñ', regex=False)
df_nomina['FUNCION'] = df_nomina['FUNCION'].str.replace('INGENIERÏ¿½A', 'INGENIERIA', regex=False)
df_nomina['FUNCION'] = df_nomina['FUNCION'].str.replace('DISEÏ¿½ADOR', 'DISEÑADOR', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('AUDITORÏ¿½A', 'AUDITORIA', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('DIRECCIÏ¿½N', 'DIRECCION', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('DIVISIÏ¿½N', 'DIVISION', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('CAPACITACIÏ¿½N', 'CAPACITACION', regex=False)
df_nomina.dropna(subset=['ANIO', 'MES_NUM', 'SUELDO_NOMINAL'], inplace=True)
df_nomina['ANIO'] = df_nomina['ANIO'].astype(int)
df_nomina['MES_NUM'] = df_nomina['MES_NUM'].astype(int)

print(f"Datos base cargados.")
display(df_nomina.head(5))
display(df_ipc_clean.head(5))

Datos base cargados.


,NOMBRE,FUNCION,DEPARTAMENTO,SUELDO,ESTATUS,MES,ANIO,SUELDO_NOMINAL,MES_NUM
0,NOEL LUPERON RAMIREZ,ASESOR ACADEMICO,CENTRO DE CAPACITACION CGR,80000,CONTRATADO,ENERO,2018,80000.0,1
1,PAOLA MARITZA POLANCO RODRIGUEZ,COORDINADOR ACADEMICO,CENTRO DE CAPACITACION CGR,40000,FIJOS,ENERO,2018,40000.0,1
2,ANA IRIS MARTINEZ NUÑEZ,DIGITADOR(A),CONSULTORIA JURIDICA,20000,CONTRATADO,ENERO,2018,20000.0,1
3,ANGEL FRANCISCO ROMAN CORCINO,AUXILIAR ADMINISTRATIVO(A),CONSULTORIA JURIDICA,25000,FIJOS,ENERO,2018,25000.0,1
4,ANNETTE ALTAGRACIA PEÑA ACOSTA,ENCARGADO DIVISION,CONSULTORIA JURIDICA,60000,FIJOS,ENERO,2018,60000.0,1


,ANIO,MES_NUM,IPC,inflacion_mensual,inflacion_acumulada,inflacion_interanual,ipc_promedio_12m
408,2018,1,94.58,0.29,0.29,3.86,3.41
409,2018,2,94.48,-0.11,0.18,3.32,3.41
410,2018,3,94.83,0.37,0.55,3.91,3.47
411,2018,4,95.21,0.40,0.95,4.05,3.51
412,2018,5,95.46,0.26,1.22,4.47,3.63


### Construcción del Data Warehouse (Esquema en Estrella)
Transformación de la tabla plana en un modelo dimensional: se generan las tablas de dimensiones (`DIM_TIEMPO`, `DIM_INSTITUCION`, `DIM_EMPLEADO`) con sus llaves primarias y se construye la tabla de hechos (`FACT_NOMINA`) con las métricas de negocio y llaves foráneas.

In [4]:
print("Iniciando construcción del Data Warehouse...")

# PREPARACIÓN DE LA BASE
fact_base = pd.merge(df_nomina, df_ipc_clean, 
                     left_on=['ANIO', 'MES_NUM'], 
                     right_on=['ANIO', 'MES_NUM'], 
                     how='left')


# ---------------------------------------------------------
# DIMENSIÓN TIEMPO
# ---------------------------------------------------------
# Identificar todos los periodos únicos presentes en los datos
dim_tiempo = fact_base[['ANIO', 'MES_NUM']].drop_duplicates().sort_values(['ANIO', 'MES_NUM']).copy()

# 2. Crear una fecha real (asumiendo día 1 de cada mes) para facilitar cálculos
# Convertimos a string 'YYYY-MM-01' y luego a datetime
dim_tiempo['fecha_temp'] = pd.to_datetime(
    dim_tiempo['ANIO'].astype(str) + '-' + dim_tiempo['MES_NUM'].astype(str) + '-01'
)

# Generar ID (YYYYMM)
dim_tiempo['id_tiempo'] = dim_tiempo['ANIO'] * 100 + dim_tiempo['MES_NUM']

# Generar Atributos Temporales
dim_tiempo['fecha'] = dim_tiempo['fecha_temp'].dt.date # Objeto fecha (sin hora)
dim_tiempo['año'] = dim_tiempo['ANIO']
dim_tiempo['mes_numero'] = dim_tiempo['MES_NUM']
dim_tiempo['mes_nombre'] = dim_tiempo['MES_NUM'].map(meses_inv_map)
dim_tiempo['dia'] = 1 # Siempre 1 porque la data es mensual

# Trimestre (1-4)
dim_tiempo['trimestre'] = dim_tiempo['fecha_temp'].dt.quarter
dim_tiempo['trimestre_nombre'] = 'T' + dim_tiempo['trimestre'].astype(str)

# Cuatrimestre (1-3)
dim_tiempo['cuatrimestre'] = ((dim_tiempo['mes_numero'] - 1) // 4) + 1
dim_tiempo['cuatrimestre_nombre'] = 'C' + dim_tiempo['cuatrimestre'].astype(str)

# Semestre (1-2)
dim_tiempo['semestre'] = np.where(dim_tiempo['mes_numero'] <= 6, 1, 2)
dim_tiempo['semestre_nombre'] = 'S' + dim_tiempo['semestre'].astype(str)

# Atributos descriptivos compuestos
dim_tiempo['anio_mes'] = dim_tiempo['año'].astype(str) + '-' + dim_tiempo['mes_numero'].astype(str).str.zfill(2)
dim_tiempo['periodo_desc'] = dim_tiempo['mes_nombre'].str[:3] + '-' + dim_tiempo['año'].astype(str)

# Limpieza final de la dimensión
cols_tiempo = [
    'id_tiempo', 'fecha', 'año', 'semestre', 'semestre_nombre', 
    'cuatrimestre', 'cuatrimestre_nombre', 'trimestre', 'trimestre_nombre', 
    'mes_numero', 'mes_nombre', 'anio_mes', 'periodo_desc'
]
dim_tiempo = dim_tiempo[cols_tiempo].reset_index(drop=True)

print(f"1. DIM_TIEMPO creada: {len(dim_tiempo)} periodos. Granularidad: Mensual.")


# ---------------------------------------------------------
# DIMENSIÓN ESTRUCTURA
# ---------------------------------------------------------
dim_estructura = fact_base[['DEPARTAMENTO', 'FUNCION']].drop_duplicates().copy()
dim_estructura.reset_index(drop=True, inplace=True)
dim_estructura['id_estructura'] = dim_estructura.index + 1

def asignar_grupo(cargo):
    cargo = str(cargo)
    if any(x in cargo for x in ['DIRECTOR', 'ENCARGADO', 'GERENTE', 'MINISTRO']): return 'ESTRATEGICO'
    if any(x in cargo for x in ['ANALISTA', 'COORDINADOR', 'SUPERVISOR', 'ABOGADO', 'AUDITOR', 'MEDICO']): return 'PROFESIONAL'
    if any(x in cargo for x in ['TECNICO', 'SOPORTE', 'AUXILIAR']): return 'TECNICO'
    return 'OPERATIVO'

dim_estructura['grupo_ocupacional'] = dim_estructura['FUNCION'].apply(asignar_grupo)
dim_estructura = dim_estructura.rename(columns={'DEPARTAMENTO': 'departamento', 'FUNCION': 'cargo'})
print(f"2. DIM_ESTRUCTURA creada: {len(dim_estructura)} posiciones.")


# ---------------------------------------------------------
# DIMENSIÓN EMPLEADO
# ---------------------------------------------------------
dim_empleado = fact_base[['NOMBRE', 'ESTATUS']].drop_duplicates(subset=['NOMBRE'], keep='last').copy()
dim_empleado.reset_index(drop=True, inplace=True)
dim_empleado['id_empleado'] = dim_empleado.index + 1
dim_empleado = dim_empleado.rename(columns={'NOMBRE': 'nombre_completo', 'ESTATUS': 'estatus_laboral'})
print(f"3. DIM_EMPLEADO creada: {len(dim_empleado)} empleados.")


# ---------------------------------------------------------
# TABLA DE HECHOS (FACT_NOMINA)
# ---------------------------------------------------------
fact_final = pd.merge(fact_base, dim_estructura, left_on=['DEPARTAMENTO', 'FUNCION'], right_on=['departamento', 'cargo'], how='left')
fact_final = pd.merge(fact_final, dim_empleado, left_on=['NOMBRE'], right_on=['nombre_completo'], how='left')
fact_final['id_tiempo'] = fact_final['ANIO'] * 100 + fact_final['MES_NUM']

# Métricas Calculadas
ipc_base_val = fact_final['IPC'].min()
fact_final['sueldo_real'] = (fact_final['SUELDO_NOMINAL'] / fact_final['IPC']) * ipc_base_val
fact_final['monto_impuestos'] = fact_final['SUELDO_NOMINAL'].apply(calcular_isr_estimado)

# SELECCIÓN FINAL DE COLUMNAS
# Aquí agregamos las nuevas métricas de inflación
fact_nomina = fact_final[[
    'id_empleado', 
    'id_tiempo', 
    'id_estructura', 
    'SUELDO_NOMINAL', 
    'monto_impuestos',
    'sueldo_real', 
    'IPC',
    'inflacion_mensual',
    'inflacion_interanual',
    'inflacion_acumulada'
]].copy()

fact_nomina.rename(columns={'SUELDO_NOMINAL': 'sueldo_nominal', 'IPC': 'valor_ipc'}, inplace=True)
fact_nomina.index.name = 'id_fact'
fact_nomina = fact_nomina.reset_index()

print(f"4. FACT_NOMINA creada con métricas económicas adicionales.")
display(fact_nomina.head(5))

Iniciando construcción del Data Warehouse...
1. DIM_TIEMPO creada: 87 periodos. Granularidad: Mensual.
2. DIM_ESTRUCTURA creada: 4322 posiciones.
3. DIM_EMPLEADO creada: 3786 empleados.
4. FACT_NOMINA creada con métricas económicas adicionales.


,id_fact,id_empleado,id_tiempo,id_estructura,sueldo_nominal,monto_impuestos,sueldo_real,valor_ipc,inflacion_mensual,inflacion_interanual,inflacion_acumulada
0,0,363,201801,1,80000.0,8583.00,79915.415521,94.58,0.29,3.86,0.29
1,1,1934,201801,2,40000.0,797.25,39957.707761,94.58,0.29,3.86,0.29
2,2,503,201801,3,20000.0,0.00,19978.853880,94.58,0.29,3.86,0.29
3,3,3187,201801,4,25000.0,0.00,24973.567350,94.58,0.29,3.86,0.29
4,4,2020,201801,5,60000.0,4195.60,59936.561641,94.58,0.29,3.86,0.29


### Almacenamiento y Exportación Final
Persistencia del modelo dimensional en una base de datos local SQLite y exportación de las tablas individuales a formato CSV para su visualización en Tableau.

In [5]:
# ---------------------------------------------------------
# EXPORTACIÓN
# ---------------------------------------------------------
db_name = 'DW_Nomina_Publica_StarSchema.db'
conn = sqlite3.connect(db_name)

dim_tiempo.to_sql('DIM_TIEMPO', conn, if_exists='replace', index=False)
dim_empleado.to_sql('DIM_EMPLEADO', conn, if_exists='replace', index=False)
dim_estructura.to_sql('DIM_ESTRUCTURA', conn, if_exists='replace', index=False)
fact_nomina.to_sql('FACT_NOMINA', conn, if_exists='replace', index=False)

conn.close()

# Exportar CSVs para Tableau/PowerBI
dim_tiempo.to_csv('DIM_TIEMPO.csv', index=False, encoding='utf-8')
dim_empleado.to_csv('DIM_EMPLEADO.csv', index=False, encoding='utf-8')
dim_estructura.to_csv('DIM_ESTRUCTURA.csv', index=False, encoding='utf-8')
fact_nomina.to_csv('FACT_NOMINA.csv', index=False, encoding='utf-8')

print(f"--- PROCESO COMPLETADO ---")

--- PROCESO COMPLETADO ---
